<a href="https://colab.research.google.com/github/JantharatChumsang/CPE378_Machine_Learning/blob/main/crawler230729.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 環境安裝

In [1]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,231 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [865 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [975 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,091 kB]
Get:13 http://security.ubun

### 引入相關套件

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import requests
import time
from selenium import webdriver
from lxml import etree
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [ ]:
# webdriver 參數配置
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(
    options=options
)

In [ ]:
def scroll_to_bottom():
    last_height = driver.execute_script('\
    return document.body.scrollHeight')
    while True:
        driver.execute_script('\
        window.scrollTo(0,document.body.scrollHeight)')
        # waiting for the results to load
        # Increase the sleep time if your internet is slow
        time.sleep(1)
        new_height = driver.execute_script('\
        return document.body.scrollHeight')
        # click on "Show more results" (if exists)
        try:
            driver.find_element_by_css_selector(".YstHxe input").click()
            # waiting for the results to load
            # Increase the sleep time if your internet is slow
            time.sleep(1)
        except:
            pass

        # checking if we have reached the bottom of the page
        if new_height == last_height:
            break

        last_height = new_height

In [ ]:
def search(query, sav_path, max_len):

  # 建立資料夾
  save_dir = os.path.join(save_path, query)
  try:
    os.makedirs(save_dir)
  except:
    # 如已存在該資料夾，就不會再做新增
    pass

  # 到google圖片搜尋的首頁
  driver.get('https://images.google.com/')
  driver.maximize_window()
  # 找到輸入框並輸入我們的關鍵字> 按enter
  submit = driver.find_element(By.CLASS_NAME, 'gLFyf')
  submit.send_keys(query)
  submit.send_keys(Keys.ENTER)

  # 滑到頁面底端
  scroll_to_bottom()

  # 爬每一張顯示的圖片
  find_elements = driver.find_elements(By.CLASS_NAME, 'rg_i')

  # 搜尋到的圖片數量
  length = min(len(find_elements), max_len)

  # counter (為每張照片命名用)
  counter=0

  for idx in range(0, length):
    try:
      find_elements[idx].click()
      time.sleep(3)
      img = driver.find_element(By.CLASS_NAME, "iPVvYb")
      images_url = img.get_attribute("src")
      print(images_url)
      reponse = requests.get(images_url)
      print(reponse)
      if reponse.status_code == 200:
        with open(os.path.join(save_dir, f"{counter+1}.jpg"),"wb") as file:
            file.write(reponse.content)
        counter+=1
      if counter==max_len:
        break
    except:
      pass

### 參數設定

In [ ]:
# 要搜尋的食物
query = ["蛤仔湯"]

# 要存放的路徑
save_path = "./drive/Shareddrives/teep2023/TeepS4/"

# 張數
max_len = 400

# 開始爬蟲
for q in query:
  search(q, save_path, max_len)


https://tokyo-kitchen.icook.network/uploads/step/cover/1259295/84bcdea3cfe1f94d.jpg
<Response [200]>
https://d3l76hx23vw40a.cloudfront.net/recipe/bk159-035a.jpg
<Response [200]>
https://img.ltn.com.tw/Upload/food/page/2017/12/16/171216-7019-0-KuPtj.jpg
<Response [200]>
https://d3l76hx23vw40a.cloudfront.net/recipe/gy61-075a.jpg
<Response [200]>
https://d3l76hx23vw40a.cloudfront.net/recipe/whk071-010b.jpg
<Response [200]>
https://tokyo-kitchen.icook.network/uploads/recipe/cover/272237/bd7cd205562624e0.jpg
<Response [200]>
https://tokyo-kitchen.icook.network/uploads/recipe/cover/209571/da95793723b3dfcb.jpg
<Response [200]>
https://tokyo-kitchen.icook.network/uploads/recipe/cover/135673/fc73019e2085a8c1.jpg
<Response [200]>
https://live.staticflickr.com/65535/51027581146_0699d7979b_b.jpg
<Response [200]>
https://img.cook1cook.com/upload/cover/201903/45/79748-1552898281_thumb.JPG
<Response [200]>
https://img.ltn.com.tw/Upload/food/page/2016/11/17/161117-4941-0-gpqlb.jpg
<Response [200]>
htt